En este ejercicio, el alumno tendrá que plantear y resolver un problema utilizando aprendizaje
automático. Se tendrán que llevar a cabo todos los pasos necesarios para la resolución del
problema: planteamiento, diseño, adquisición de los datos, análisis, entrenamiento de
algoritmos, evaluación, discusión de resultados, etc. Todo este proceso se desarrollará en un
cuaderno de Jupyter que conformará la entrega final.
El problema planteado podrá ser inventado o real, y puede estar basado en un problema de
negocio, un artículo científico, una competición, etc. Existen únicamente dos requisitos en
cuanto al caso de uso:
1. Tiene que utilizarse un conjunto de datos para su resolución.
2. La solución debe estar basada en aprendizaje automático, es decir, se debe haber
entrenado al menos un algoritmo.

In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing as prep
from sklearn.metrics import precision_score, recall_score, confusion_matrix, mean_absolute_error, mean_absolute_percentage_error, r2_score, accuracy_score, mean_squared_error
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import utils
from sklearn.svm import SVR

ModuleNotFoundError: No module named 'pandas'

In [3]:
print("Hello world!")

Hello world!
